In [73]:
#from elasticsearch import Elasticsearch
import os
import json
from jsmin import jsmin
import pyelasticsearch
from pyelasticsearch import ElasticSearch #see if this is better than 'official API' - the docs look better to me
from pyelasticsearch import bulk_chunks


In [2]:
%install_ext https://raw.githubusercontent.com/szeitlin/watermark/master/watermark.py

Installed watermark.py. To use it, type:
  %load_ext watermark


In [3]:
%load_ext watermark

In [47]:
%watermark -a "Samantha Zeitlin" -d -u -p pyelasticsearch,jsmin,python 

Samantha Zeitlin 
Last updated: 04/11/2015 

pyelasticsearch 1.2.1
jsmin 2.1.1
python 2.7.9


In [69]:
try:
    es = ElasticSearch('http://localhost:9200')
except:
    ConnectionError
    print "I can't connect!"

In [86]:
#start fresh to see if mapping first is better than mapping after
#es.delete_all_indexes() #the deletion worked, but then it gave an error b/c there was nothing to delete. 
#note that there doesn't seem to be a method for _status? maybe use send_request?
#print ElasticSearch.send_request.__doc__
es.send_request("GET", ["_status"],{},{}) #This didn't work because it wants an iterable (but apparently string is not what they mean??)

{u'_shards': {u'failed': 0, u'successful': 19, u'total': 35},
 u'indices': {u'.kibana': {u'docs': {u'deleted_docs': 0,
    u'max_doc': 4,
    u'num_docs': 4},
   u'flush': {u'total': 1, u'total_time_in_millis': 1},
   u'index': {u'primary_size_in_bytes': 14282, u'size_in_bytes': 14282},
   u'merges': {u'current': 0,
    u'current_docs': 0,
    u'current_size_in_bytes': 0,
    u'total': 0,
    u'total_docs': 0,
    u'total_size_in_bytes': 0,
    u'total_time_in_millis': 0},
   u'refresh': {u'total': 1, u'total_time_in_millis': 0},
   u'shards': {u'0': [{u'docs': {u'deleted_docs': 0,
       u'max_doc': 4,
       u'num_docs': 4},
      u'flush': {u'total': 1, u'total_time_in_millis': 1},
      u'index': {u'size_in_bytes': 14282},
      u'merges': {u'current': 0,
       u'current_docs': 0,
       u'current_size_in_bytes': 0,
       u'total': 0,
       u'total_docs': 0,
       u'total_size_in_bytes': 0,
       u'total_time_in_millis': 0},
      u'refresh': {u'total': 1, u'total_time_in_mill

In [75]:
!pwd

/Users/szeitlin/mystuff/projects/nbindex/nbindex


In [85]:
#put a few target files in folder and iterate through that)
#with open("elk_test/import-one-year.ipynb", 'r') as test:
#    minified = jsmin(test.read())

path = '/Users/szeitlin/mystuff/projects/nbindex/nbindex/testdir'
def notebooks():
    #os.listdir(path)
    for eachfile in os.listdir(path):
        fullname = os.path.join(path,eachfile)
        with open(fullname, 'r') as each:
            yield jsmin(each.read())
            
for chunk in bulk_chunks(notebooks()):
    es.bulk(chunk, doc_type='notebook', index='test')


ElasticHttpError: Non-OK response returned (400): u'ActionRequestValidationException[Validation Failed: 1: no requests added;]'

In [71]:
#looks like it wants you to create an index first and put_mapping second
print es.put_mapping.__doc__

with open("mapping_minus_curl.json", 'r') as map:
    minimap = jsmin(map.read())
es.put_mapping(test, 'notebook', minimap)


        Register specific mapping definition for a specific type against one or
        more indices.

        :arg index: An index or iterable thereof
        :arg doc_type: The document type to set the mapping of
        :arg mapping: A dict representing the mapping to install. For example,
            this dict can have top-level keys that are the names of doc types.
        :arg ignore_conflicts: See the ES docs.

        See `ES's put-mapping API`_ for more detail.

        .. _`ES's put-mapping API`:
            http://www.elastic.co/guide/en/elasticsearch/reference/current/indices-put-mapping.html
        


ElasticHttpNotFoundError: Non-OK response returned (404): u'IndexMissingException[[_status] missing]'

In [64]:
#replace this with bulk_chunks instead 
#es.bulk([es.index_op(minified), es.index_op(mini2)], doc_type='notebook', index='test2')

In [66]:
!ls

Dockerfile                                 another_query.txt                          first_try_mapping.json                     minified_testfile.json                     testcreate.json
LICENSE                                    draft_bulk_import_multiple_notebooks.ipynb import_statements_query.txt                pyminified_testfile.json                   testnb.json
README.md                                  elasticsearch.yml                          match_query.txt                            test2.json                                 testprefix.json
action.json                                elk_test                                   minified.json                              testbulk.json                              testquery.txt


In [28]:
with open('import_statements_query.txt', 'r') as query:
    q=query.read()
    es.search(q, index='test2')

In [21]:
%%bash
curl -s -XPOST localhost:9200/_bulk --data-binary @test2.json


{"took":251,"errors":false,"items":[{"index":{"_index":"test","_type":"type1","_id":"2","_version":1,"status":201}}]}

In [ ]:
#looks like kibana barely works at all with only 1 document, suddenly with 2 it starts showing more functionality. 